<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zur Umsetzung in der Bachelorarbeit müssen noch einige Sachen beachtet werden:


*   Beim Abrufen der Startseite von Yahoo Finance News wird das HTML Dokument geladen, welches noch unvollständig ist, da Javascript noch nicht gelaufen ist.
Das führt dazu, dass wichtige Elemente nicht im Dokument vorhanden sind (in diesem Beispiel gibt es ein div-Element mit dem Inhalt "Business", wodurch man weiß, ob der Artikel relevant ist oder nicht und dieser kommt erst mit Javascript). Eine Lösung wäre, das Scraping mit Selenium vorzunehmen, da damit auch das Javascript laufen gelassen werden kann. Damit wird es auch möglich sein auf der Seite runterzuscrollen, um ältere Nachrichten zu laden und scrapen.
Sobald man die einzelnen Links zu den Nachrichten hat, kann man diese Liste weiter an den Code unten geben.



Hole HTML-Dokument der Nachricht über dessen Link und ziehe dort das div-Element mit dem Nachrichtentext raus.

In [ ]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://finance.yahoo.com/news/global-markets-shares-gain-dollar-190503894.html")

soup = BeautifulSoup(res.text, "html.parser")

title = soup.find("title").text
print(title)

soup = soup.find("div", attrs={'class':'caas-body'})

GLOBAL MARKETS-Shares gain, dollar falls with Fed comments inspiring dovish bets


Hier wird weiter gearbeitet mit dem Div-Element, dass den Nachrichtentext enthält. Hierbei wird der pure Text extrahiert und Listen, Buttons und die Tags selber entfernt.

In [ ]:
for div in soup.find_all('div'):
    div.unwrap()

for ul in soup.find_all('ul'):
    # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
    p_tag = ul.find_previous_sibling('p')

    # Wenn ein <p>-Tag gefunden wurde, entferne es
    if p_tag:
        p_tag.decompose()

# Entferne alle <div>-Elemente
for ul in soup.find_all("ul"):
    ul.decompose()

for button in soup.find_all("button"):
    button.decompose()

for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
    p_tag.decompose()

# text_lines = [p.get_text() + '\n' for p in soup.find_all('p')]

for p in soup.find_all('p'):
  print(p.text)

# print(text_lines)

(Updates at 01:39 p.m ET/1839 GMT)
By Sinéad Carew and Amanda Cooper
NEW YORK/LONDON, Dec 1 (Reuters) - MSCI's global stock index gained ground on Friday, while U.S. Treasury yields and the dollar were lower after Federal Reserve Chair Jerome Powell vowed to move "carefully" on interest rates.
Treasury yields fell after Powell said the risks of hiking interest rates too much and slowing the economy more than necessary have become "more balanced" with the risks of not hiking enough to control inflation.
"Powell is trying to be balanced, trying to make sure the market doesn't get ahead of itself. He doesn't want the market or traders to speculate on rate decreases," said Tim Ghriskey, senior portfolio strategist at Ingalls & Snyder in New York.
"He's all about the data, and the core inflation data over the last six months has been good. But he reiterates the objective is still 2% and he doesn't want all the work the Fed has done to bring inflation down to suddenly be reversed."
But while

Klassifiziere die Nachricht mit einem vortrainierten Modell mit der Huggingsface Bibliothek und PyTorch

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Dein Text
text = ""

for p in soup.find_all('p'):
  text = text + p.text + "\n"
# Tokenisiere den Text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Wende das Modell an
with torch.no_grad():
    outputs = model(**inputs)

# Hole die Modellvorhersagen
predictions = outputs.logits

# Konvertiere Vorhersagen in Wahrscheinlichkeiten und hole das maximale
predictions = torch.nn.functional.softmax(predictions, dim=1)
predicted_class = torch.argmax(predictions).item()

# Zuordnung der Klassen
classes = ["bearish", "neutral", "bullish"]

# Ergebnis
print(f"Sentiment: {classes[predicted_class]}")


Sentiment: bullish
